# Read the ASHE data

ASHE is the Annual Survey of Hours and Earnings. It contains information about salaries in various regions and sectors of the UK. Here we want to calculate median salaries in Nesta sectors which we can then use to benchmark industries.


**Tasks**

* Scrape the data
* Parse it
* Merge with out segments at the finest level of granularity possible
* Create weighted median taking into account distribution of employment in the segment

## Preamble

In [1]:
%run ../notebook_preamble.ipy

In [2]:
from zipfile import ZipFile
from io import BytesIO

In [3]:
def get_ashe_data(path,ons_path):
    '''
    Function to collect the ASHE data from the ONS website.
    
    Arguments:
        path (str) is the path for the file we are interested in
        ons_path (str) is the parent for all ashe files
        
    This will return a doanloaded and parsed file
    
    '''
    
    file = requests.get(ons_path+path)
    
    #Create a zipfile with its content
    z = ZipFile(BytesIO(file.content))
    
    #Extract names
    names = z.namelist()
    
    #Select the names (they will meantion hourly gross but not the confidence intervals)
    
    my_name = [x for x in names if (all(names in x for names in ['Annual','Gross'])) & ('CV' not in x)]
    
    print(my_name)

    #if len(my_name)>1:
    #    print('Too many options')
    #    break
    
    #Read into excel
    infile = pd.read_excel(BytesIO(z.open(my_name[0]).read()),sheet_name=1,skiprows=4,
                      na_values=['x','..',':'])
    
    #Drop missing values in the matching code or median (these don't interest us)
    infile.dropna(axis=0,subset=['Code'],inplace=True)
    
    infile['Code'] = [x.strip() for x in infile['Code']]
    
    #container.append(infile.reset_index(drop=True))
    
    return(infile.reset_index(drop=True))

In [4]:
#Processing files
def add_zeros(container):
    '''
    This adds pre-zeroes to codes in categories A and B
    
    Args:
        Container (df) is one of the dataframes we have created before
    
    '''
    
    new_cont = container.copy()
    
    for pid,row in new_cont.iterrows():
        
        if row['Code']=='C':
            break
        else:
            if row['Code'] not in ['A','B']:
            
            #print(row['Code'])
                new_cont.loc[pid,'Code']='0'+row['Code']
        
    return(new_cont)
    

def year_ashe_lookups(ashe_table):
    '''
    
    Takes an ashe table and outputs a list of code lookups depending on the level of resolution at which they are available
    
    Args:
        ashe_table: an ashe table as above
        
    returns three dicts with code - salary lookups with decreasing levels of resolution
    
    '''
    
    #Containers
    ashe_4 = {}
    ashe_3 = {}
    ashe_2 = {}

    #In each row it gets the length of a code (sic4,3 etc) and assigns the median salary to the right dict.
    #We we will use this later to assign the median to 
    
    for pid, row in ashe_table.iterrows():

        code = row['Code'].strip()
        med_sal = row['Median']

        if len(code)==4:
            ashe_4[code]= med_sal

        elif len(code)==3:
            ashe_3[code] = med_sal

        elif len(code)==2:
            ashe_2[code] = med_sal

        else:
            pass
        
    return([ashe_4,ashe_3,ashe_2])
    
    

In [5]:
def map_salaries(lookup,four_digit, ashe):
    '''
    
    Assigns each 4-digit sic code a median according to ASHE at its finest level of resolution
    
    Args:
        lookup (df) a lookup with the 4 digit sic code we want to query against our ashe lookups
        four_digit (str) the name of the variable with the four digits
        ashe_lookups (list of dicts) the list of ashe code-median key-value pairs to query
        
    Returns
        a table with four digit sics, names and salaries.
    
    
    '''
    
    cont = []

    #Is loo
    for sic in lookup[four_digit]:
        
        if sic in ashe[0].keys():
            #cont.append({sic:ashe_lookups[0][sic]})
            cont.append([sic,ashe[0][sic]])
            
        elif sic[:-1] in ashe[1].keys():
            
            #cont.append({sic:ashe_lookups[1][sic[:-1]]})
            cont.append([sic,ashe[1][sic[:-1]]])
        
        elif sic[:-2] in ashe[2].keys():
            #cont.append({sic:ashe_lookups[2][sic[:-2]]})
            cont.append([sic,ashe[2][sic[:-2]]])
        
        else:
            #cont.append({sic:np.nan})
            cont.append([sic,np.nan])
    
    return(pd.DataFrame(cont,columns=['sic_4','median_salary_thGBP']).set_index('sic_4'))
        
    
    

## 1. Collect data

In [6]:
standard_path = 'https://www.ons.gov.uk/file?uri=/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/industry4digitsic2007ashetable16/'

#Ashe paths
ashe_paths = ['2018provisional/table162018provisional.zip', '2017revised/table162017revised.zip',
            '2016revised/table162016revised.zip','2015/table162015revised.zip']

In [7]:
ashes = [get_ashe_data(p,standard_path) for p in ashe_paths]

['PROV - SIC07 Industry (4) SIC2007 Table 16.7a   Annual pay - Gross 2018.xls']
['SIC07 Industry (4) SIC2007 Table 16.7a   Annual pay - Gross 2017.xls']
['SIC07 Industry (4) SIC2007 Table 16.7a   Annual pay - Gross 2016.xls']
['SIC07 Industry (4) SIC2007 Table 16.7a   Annual pay - Gross 2015.xls']


We have lost a few codes with zero at the beginning

In [8]:
ashes[0].head()['Code']

0      A
1      1
2     11
3    111
4    112
Name: Code, dtype: object

In [9]:
new_containers = [add_zeros(x) for x in ashes]

### Cluster lookup

In [11]:
cl = pd.read_csv('../../data/raw/sic_4_industry_segment_lookup.csv',dtype={'sic_4':str})

## 2. Processing

We need to assign salaries to their more detailed code. If a salary is available at the 4-digit then we are not interested in the salary at the 3 digit

### Create 2 digit and four digit lookups from Ashe

In [12]:
all_ashe_lookups = [year_ashe_lookups(cont) for cont in new_containers]

In [13]:
all_salaries = pd.concat([map_salaries(cl,'sic_4',tab) for tab in all_ashe_lookups],axis=1)

all_salaries.columns = [2018,2017,2016,2015]

In [14]:
all_salaries.head()

,2018,2017,2016,2015
sic_4,,,,
4331,NaN,NaN,26295.0,NaN
4312,29443.0,NaN,NaN,29433.0
4211,32997.0,30638.0,30072.0,31336.0
4120,31820.0,31025.0,29472.0,27763.0
4399,29511.0,30480.0,29675.0,28649.0


#### Create weighted medians

We are not going to be working with sic codes, but our own segments. This requires creating weighted medians of salaries across SIC codes. We use levels of employment to create the weights.


In [15]:
#Melt the salaries file from above
salaries_long = all_salaries.reset_index(drop=False).melt(id_vars=['sic_4'],var_name='year',value_name='median_salary')

In [17]:
#We read for the four years
bres_data = pd.concat([pd.read_csv(f'../../data/interim/industry/nomis_BRES_{y}_TYPE450.csv',dtype={'SIC4':str}) for y in [2016,2017,2018]],axis=0)

bres_data.head()

,Unnamed: 0,year,geo_type,geo_nm,geo_cd,SIC4,value,cluster_name
0,0,2016,nuts 2013 level 2,Tees Valley and Durham,UKC1,0161,100,manufacture_food
1,1,2016,nuts 2013 level 2,Northumberland and Tyne and Wear,UKC2,0161,200,manufacture_food
2,2,2016,nuts 2013 level 2,Cumbria,UKD1,0161,200,manufacture_food
3,3,2016,nuts 2013 level 2,Greater Manchester,UKD3,0161,350,manufacture_food
4,4,2016,nuts 2013 level 2,Lancashire,UKD4,0161,500,manufacture_food


In [18]:
#Group them by year to get the total level of employment by SIC4
sic_yearly_long = bres_data.groupby(['year','SIC4'])['value'].sum().reset_index(drop=False)

sic_yearly_long.rename(columns={'value':'employment'},inplace=True)

In [19]:
salary_empl_merge = pd.merge(salaries_long,sic_yearly_long,left_on=['sic_4','year'],right_on=['SIC4','year'])

In [20]:
segment_merged = pd.merge(cl[['sic_4','cluster']],salary_empl_merge,left_on='sic_4',right_on='sic_4')

In [21]:
segment_merged.head()

,sic_4,cluster,year,median_salary,SIC4,employment
0,4331,construction_construction,2018,NaN,4331,13970
1,4331,construction_construction,2017,NaN,4331,14620
2,4331,construction_construction,2016,26295.0,4331,15025
3,4312,construction_construction,2018,29443.0,4312,14130
4,4312,construction_construction,2017,NaN,4312,15110


In [22]:
#Weighted salary: takes all the sics in a segment and applies a weight based on their importance in the segment
weighted_sal = segment_merged.groupby(
    ['cluster','year']).apply(lambda x: np.sum(x['median_salary']*x['employment'])/np.sum(x['employment'])).reset_index(drop=False)

In [23]:
ashe_out = weighted_sal.rename(columns={0:'weighted_median_salary'})

ashe_out.pivot_table(index='cluster',columns='year',values='weighted_median_salary').corr()

year,2016,2017,2018
year,,,
2016,1.000000,0.751622,0.856215
2017,0.751622,1.000000,0.863850
2018,0.856215,0.863850,1.000000


**Remove some outliers**

In [24]:
for pid,row in ashe_out.iterrows():
    
    if row['weighted_median_salary']<1000:
    
        ashe_out.loc[pid,'weighted_median_salary'] = np.nan
    

**Calculate averages for all years**

In [25]:
ashe_out_grouped = pd.DataFrame(ashe_out.groupby(['cluster'])['weighted_median_salary'].mean())

In [26]:
ashe_out_grouped['ashe_median_salary_rank'] = pd.qcut(ashe_out_grouped['weighted_median_salary'],np.arange(0,1.1,0.1),labels=False)

In [27]:
ashe_out_grouped.sort_values('ashe_median_salary_rank',ascending=False).tail()

,weighted_median_salary,ashe_median_salary_rank
cluster,,
services_textiles,13508.121575,0.0
services_hospitality,12426.698110,0.0
services_electronics_machinery,7636.231977,0.0
manufacture_textiles,12078.750425,0.0
manufacture_light,NaN,NaN


In [29]:
ashe_out_grouped.to_csv(f'../../data/interim/industry/{today_str}_ashe_rankings.csv')